## Description:

Compare OTUs that vary in rhizosphere and vary in bare soil by date and test for correlation


# Setting variables

In [1]:
# workDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/DeSeq2/'
# respFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/DeSeq2/Responders.txt'
#using physeq file with sparsity of greater than 3 in 3 samples
physeqFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/physeq/Full-Sparsity3in3'

biomFileDir = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/OTU_binning/'

biomFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/OTU_binning/otu_table_wtax.biom'
metadataFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/metadata_RhizCG_merged.txt'
treeFile = '/home/bryan/RhizCG/data/MiSeq_amplicon/MergedRuns/fasttree/otusn.tree'

# Init

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(ggplot2)
library(phyloseq)
library(tidyr) 
library(plyr)
library(dplyr)
library(scales)
library(biom)
library(gridExtra)
library(metagenomeSeq) 
library(doParallel)





/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/f

## Loading MRexperiment object

In [5]:
%%R -i physeqFile

physeq.Full = readRDS(physeqFile)
physeq.Full = physeq.Full %>% filter_taxa(function(x) sum(x) > 0, TRUE)
physeq.Full

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 4982 taxa and 238 samples ]
sample_data() Sample Data:       [ 238 samples by 55 sample variables ]
tax_table()   Taxonomy Table:    [ 4982 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 4982 tips and 4981 internal nodes ]


In [ ]:
physeqFile

In [ ]:
%%R
test = prune_taxa(c("OTU.93", "OTU.94"), physeq.Full) 
phyloseq::plot_bar(test, x = "Planted")

# Do bare soil samples change with date?

## Convert to MRexperiment

In [ ]:
%%R
sample_data(physeq.Full)$DAP_rel = sample_data(physeq.Full)$DAP - 36

### Create dummy ID for rhizosphere samples

In [ ]:
%%R
sample_data(physeq.Full)$Planted.Rep = as.factor(paste(sample_data(physeq.Full)$Planted, 
                                                       sample_data(physeq.Full)$Rep, sep = "_" ))

sample_data(physeq.Full)$Planted.Rep.DAP = as.factor(paste(sample_data(physeq.Full)$Planted.Rep, 
                                                       sample_data(physeq.Full)$DAP, sep = "_" ))


In [ ]:
%%R
#Merge based on Planted.Rep
phy.merged = merge_samples(physeq.Full, "Planted.Rep.DAP")

#Create new variable names
sample_data(phy.merged)$Sample = sample_names(phy.merged)

#pull Planted and rep status from sample names
    sample_data(phy.merged)$Planted.Rep = substr(sample_data(phy.merged)$Sample, 1, 6)
    sample_data(phy.merged)$Planted.Rep[33:64] = substr(sample_data(phy.merged)$Sample[33:64], 1, 13)

    # convert to factor
    sample_data(phy.merged)$Planted.Rep = as.factor(sample_data(phy.merged)$Planted.Rep)

#assign planted status
    sample_data(phy.merged)$Planted[1:32] = "BARE"
    sample_data(phy.merged)$Planted[33:64] = "Rhizosphere"
    sample_data(phy.merged)$Planted = as.factor(sample_data(phy.merged)$Planted)
#OTU table is transposed in merge, not sure why, but re-transpose in case it is interfering with downstream commands
otu_table(phy.merged) = t(otu_table(phy.merged))

In [ ]:
%%R

make_metagenomeSeq = function(physeq) {
    require("metagenomeSeq")
    require("phyloseq")
    # Enforce orientation
    if (!taxa_are_rows(physeq)) {
        physeq <- t(physeq)
    }
    OTU = as(otu_table(physeq), "matrix")
    #OTUTill = subset(OTU, rownames(OTU) %in% df.Till.r$OTU)

    # Convert sample_data to AnnotatedDataFrame
    ADF = AnnotatedDataFrame(data.frame(sample_data(physeq)))
    # define dummy 'feature' data for OTUs, using their name Helps with
    # extraction and relating to taxonomy later on.
    TDF = AnnotatedDataFrame(data.frame(tax_table(physeq)))
    #TDFTill = subset(TDF, rownames(TDF) %in% df.Till.r$OTU)
    TDF$Rank9 = rownames(TDF)


    # Create the metagenomeSeq object
    MGS = newMRexperiment(counts = OTU, phenoData = ADF, featureData = TDF)
    # Trigger metagenomeSeq to calculate its Cumulative Sum scaling factor.
    MGS = cumNorm(MGS)
    return(MGS)
}

MR = make_metagenomeSeq(physeq.Full)

In [ ]:
%%R
str(MR)

In [ ]:
%%R
TimeSeries = function(MR, feature) {  
    
    res = fitTimeSeries(obj = MR, lvl = 'Rank9', feature = feature, class = "Planted",
                    id = "Planted.Rep", time = "DAP_rel", log = TRUE, B = 10)

    return(res)
}




In [ ]:
%%R
N10 = sort(taxa_sums(physeq.Full),TRUE)[1:100] %>% names
physeq.t10 <- prune_taxa(N10, physeq.Full)
physeq.t10

In [ ]:
%%R
tax_table(phy.merged) %>% head

In [ ]:
%%R
tax_table(phy.merged) %>% as.matrix %>% as.data.frame %>% filter(rownames(.) == "OTU.5548" | rownames(.) == "OTU.80")

In [ ]:
%%R
sample_data(test) %>% head

In [ ]:
%%R
prune_taxa(c("OTU.93"), physeq.Full) %>% plot_bar(x = "Planted.Rep.DAP")



In [ ]:
%%R
prune_taxa(c("OTU.94"), physeq.Full) %>% plot_bar(x = "Planted.Rep.DAP")

In [ ]:
%%R
subset_taxa(physeq.Full, Rank2 == "Chloroflexi") %>% plot_tree()


In [ ]:
%%R
plot_bar(Chlor, x = "Rep")

In [ ]:
%%R
(MRcounts(MR)) %>% as.matrix()  %>% as.data.frame %>% filter(rownames(.) == feature[1] | rownames(.) == feature[5]) 

In [ ]:
%%R
# subset to 200 most abundant
# this should be done on relative abundance data as

OTU = as(otu_table(physeq.t10), "matrix")


feature = rownames(OTU)
str(feature)

In [ ]:
%%R


In [ ]:
%%R
test = prune_taxa(feature[73:74], phy.merged)
test %>% otu_table

In [ ]:
%%R
#OTUs not found across all timepoints?
test = OTU %>% apply(1,sum)
range(test)

In [ ]:
%%R
res1 = fitTimeSeries(obj = MR, lvl = 'Rank9', feature = feature[5], class = "Planted",
                    id = "Planted.Rep", time = "DAP", log = TRUE, B = 10)

In [ ]:
%%R
res2 = fitTimeSeries(obj = MR, lvl = 'Rank9', feature = feature[1], class = "Planted",
                    id = "Planted.Rep", time = "DAP", log = TRUE, B = 10)

In [ ]:
%%R
res1

In [ ]:
%%R
res

In [ ]:
%%R
?fitTimeSeries

In [ ]:
%%R
# apply fit time series across features in row
registerDoParallel(20)

TS = llply(feature, 
          TimeSeries, 
          .parallel = TRUE,
          MR = MR 
         )

In [ ]:
%%R
names(TS) = feature

In [ ]:
%%R
timeSeriesFits = sapply(TS,function(i){i[[1]]})[-grep("No",TS)]

In [ ]:
%%R
str(timeSeriesFits)

In [ ]:
%%R
for(i in 1:length(timeSeriesFits)){
    rownames(timeSeriesFits[[i]]) =
    paste(
    paste(names(timeSeriesFits)[i]," interval",sep=""),
    1:nrow(timeSeriesFits[[i]]),sep=":"
)
}

In [ ]:
%%R
timeSeriesFits = as.data.frame(do.call(rbind, timeSeriesFits))


#do.call(rbind,timeSeriesFits)

In [ ]:
%%R
pvalues = timeSeriesFits[,"p.value"]
adjPvalues = p.adjust(pvalues,"bonferroni")
timeSeriesFits = cbind(timeSeriesFits,adjPvalues)
head(timeSeriesFits)

In [ ]:
%%R
write.csv(timeSeriesFits, 'data/timeSeries_OM_ITS.csv')

In [ ]:
%%R
TSOM = read.csv('data/timeSeries_OM_ITS.csv')

In [ ]:
%%R
# calculate weighted and unweighte uni-frac distances
registerDoParallel(cores=28)
bare.wunif.dist = distance(physeq.Rb, 
                      method = "unifrac", 
                      weighted = TRUE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

bare.unif.dist = distance(physeq.Rb, 
                      method = "unifrac", 
                      weighted = FALSE,
                      fast = TRUE, 
                      parallel = TRUE, 
                      normalized = FALSE)

In [ ]:
%%R -w 800 -h 350
# Generate NMDS for 
sample_data(physeq.Rb)$DAP = factor(sample_data(physeq.Rb)$DAP)

sample_data(physeq.Rb)$Rep = factor(sample_data(physeq.Rb)$Rep)

nmds_w = ordinate(physeq.Rb, 
                method = "NMDS", 
                distance = bare.wunif.dist)

nmds_u = ordinate(physeq.Rb, 
                method = "NMDS", 
                distance = bare.unif.dist)


In [ ]:
%%R -w 500 -h 600
p = plot_ordination(physeq.Rb, nmds_w, justDF = TRUE)
p_u = plot_ordination(physeq.Rb, nmds_u, justDF = TRUE)

## generating plots
p1 = ggplot(p, aes(x = NMDS1, y = NMDS2)) +
        geom_point(aes(fill=DAP), pch = 21, size = 4) +
        ggtitle("NMDS of weighted-unifrac distance \n Bare Soil Samples")
        #scale_size(range=c(2,8))


p2 = ggplot(p_u, aes(x = NMDS1, y = NMDS2)) +
       geom_point(aes(fill=DAP), pch=21, size = 4) +
        ggtitle("NMDS of unweighted-unifrac distance \n Bare Soil Samples")
        #scale_size(range=c(2,8))

grid.arrange(p1, p2, ncol=1)

In [ ]:
%%R
set.seed(1)

df = as(sample_data(physeq.Rb), "data.frame")

#weighted unifrac
d = bare.wunif.dist
Bareadonis = adonis(formula = d ~ Library + Treatment + DAP, df, strata = df$Rep, permutations = 999)
print("PERMANOVA of bare soil samples with w-unifrac")
Bareadonis %>% print

#weighted unifrac
d = bare.unif.dist
Bareadonis = adonis(formula = d ~ Library + Treatment + DAP, df, strata = df$Rep, permutations = 999)
print("PERMANOVA of bare soil samples with unifrac")
Bareadonis %>% print




* MCC in bare soil samples change by date in both weighted and un-weighted unifrac estimates
* Effect is robust to inclusion of library in permanova

## Filter taxa to rhizosphere responders


In [ ]:
%%R -i respFile
Responders = read.table(respFile, header = FALSE, sep = "\t")
colnames(Responders) = c("OTU", "Resp")
Responders = filter(Responders, Resp == 1, TRUE)

In [ ]:
%%R
Rsel = as.character(Responders[,1])
head(Rsel)

In [ ]:
%%R

#re-order levels 
sample_data(physeq.Full)$Plant = relevel(sample_data(physeq.Full)$Plant, "BARE")

#Set DAP as factor
sample_data(physeq.Full)$DAP = factor(sample_data(physeq.Full)$DAP) 
sample_data(physeq.Full)$Rep = factor(sample_data(physeq.Full)$Rep) 
sample_data(physeq.Full)$DAP %>% levels %>% print

#sample_data(physeq.Full)$PlantRep = interaction(sample_data(physeq.Full)$Rep, sample_data(physeq.Full)$Plant)


In [ ]:
%%R
physeq.Resp = prune_taxa(Rsel, physeq.Full)
physeq.Resp = subset_samples(physeq.Resp, Library == 1, TRUE)
sample_data(physeq.Resp)$DAP %>% levels %>% print
sample_data(physeq.Resp)$Plant %>% levels %>% print

In [ ]:
%%R
## Create phyloseq object on subset of samples
physeq.plant = subset_samples(physeq.Resp, Plant != "BARE")
sample_data(physeq.plant)$PlantRep = interaction(sample_data(physeq.plant)$Rep, sample_data(physeq.plant)$Plant)

In [ ]:
%%R
physeq.bare = subset_samples(physeq.Resp, Plant == "BARE")
sample_data(physeq.bare)$DAPRep = interaction(sample_data(physeq.bare)$Rep, sample_data(physeq.bare)$DAP)
physeq.bare

# still need to try removing library 2

# Create DESeq object and view results for Bare and Planted Seperately

## Test for log2fold change by date from 72 as benchmark

In [ ]:
%%R
dds = phyloseq_to_deseq2(physeq.bare, ~Rep + Treatment + DAP)
d_dds = DESeq(dds, parallel = TRUE)
#B.88 = results(d_dds, contrast = c("DAP", "72", "88"))
#B.84 = results(d_dds, contrast = c("DAP", "72", "84"))
B.79 = results(d_dds, contrast = c("DAP", "72", "79"))
B.61 = results(d_dds, contrast = c("DAP", "72", "61"))
B.57 = results(d_dds, contrast = c("DAP", "72", "57"))
B.53 = results(d_dds, contrast = c("DAP", "72", "53"))
B.36 = results(d_dds, contrast = c("DAP", "72", "36"))


In [ ]:
%%R
dds = phyloseq_to_deseq2(physeq.plant, ~Rep + Treatment + DAP)
d_dds = DESeq(dds, parallel = TRUE)
#R.88 = results(d_dds, contrast = c("DAP", "72", "88"))
#R.84 = results(d_dds, contrast = c("DAP", "72", "84"))
R.79 = results(d_dds, contrast = c("DAP", "72", "79"))
R.61 = results(d_dds, contrast = c("DAP", "72", "61"))
R.57 = results(d_dds, contrast = c("DAP", "72", "57"))
R.53 = results(d_dds, contrast = c("DAP", "72", "53"))
R.36 = results(d_dds, contrast = c("DAP", "72", "36"))

## Join tables and graph

### Days

In [ ]:
%%R
#join results tables in a single table
R = R.36
B = B.36
colnames(R) = paste("Rhiz_", colnames(R), sep ="")
colnames(B) = paste("Bare_", colnames(B), sep ="")
R = as.data.frame(R[,c(1,2,5,6)])
B= as.data.frame(B[,c(1,2,5,6)])
R$OTU = row.names(R)
B$OTU = row.names(B)
Tbl = inner_join(B, R, by = "OTU")                                   

# Graph
Tbl.f = filter(Tbl, Rhiz_baseMean > 30 , TRUE)
p.36 = ggplot(Tbl.f, aes(x = Bare_log2FoldChange, y = Rhiz_log2FoldChange)) +
    geom_point()+
    stat_smooth(method = "lm") +
ggtitle("72 vs 36")



In [ ]:
%%R
#join results tables in a single table
R = R.53
B = B.53
colnames(R) = paste("Rhiz_", colnames(R), sep ="")
colnames(B) = paste("Bare_", colnames(B), sep ="")
R = as.data.frame(R[,c(1,2,5,6)])
B= as.data.frame(B[,c(1,2,5,6)])
R$OTU = row.names(R)
B$OTU = row.names(B)
Tbl = inner_join(B, R, by = "OTU")                                   

# Graph
Tbl.f = filter(Tbl, Bare_baseMean > 20 , TRUE)
p.53 = ggplot(Tbl.f, aes(x = Bare_log2FoldChange, y = Rhiz_log2FoldChange)) +
    geom_point()+
    stat_smooth(method = "lm") +
    ggtitle("72 vs 53")


In [ ]:
%%R
#join results tables in a single table
R = R.57
B = B.57
colnames(R) = paste("Rhiz_", colnames(R), sep ="")
colnames(B) = paste("Bare_", colnames(B), sep ="")
R = as.data.frame(R[,c(1,2,5,6)])
B= as.data.frame(B[,c(1,2,5,6)])
R$OTU = row.names(R)
B$OTU = row.names(B)
Tbl = inner_join(B, R, by = "OTU")                                   

# Graph
Tbl.f = filter(Tbl, Bare_baseMean > 20 , TRUE)
p.57 = ggplot(Tbl.f, aes(x = Bare_log2FoldChange, y = Rhiz_log2FoldChange)) +
    geom_point()+
    stat_smooth(method = "lm") +
    ggtitle("72 vs 57")


In [ ]:
%%R
#join results tables in a single table
R = R.61
B = B.61
colnames(R) = paste("Rhiz_", colnames(R), sep ="")
colnames(B) = paste("Bare_", colnames(B), sep ="")
R = as.data.frame(R[,c(1,2,5,6)])
B= as.data.frame(B[,c(1,2,5,6)])
R$OTU = row.names(R)
B$OTU = row.names(B)
Tbl = inner_join(B, R, by = "OTU")                                   

# Graph
Tbl.f = filter(Tbl, Bare_baseMean > 20 , TRUE)
p.61 = ggplot(Tbl.f, aes(x = Bare_log2FoldChange, y = Rhiz_log2FoldChange)) +
    geom_point()+
    stat_smooth(method = "lm") + 
    ggtitle("Day 72 vs 61")


In [ ]:
%%R
#join results tables in a single table
R = R.79
B = B.79
colnames(R) = paste("Rhiz_", colnames(R), sep ="")
colnames(B) = paste("Bare_", colnames(B), sep ="")
R = as.data.frame(R[,c(1,2,5,6)])
B= as.data.frame(B[,c(1,2,5,6)])
R$OTU = row.names(R)
B$OTU = row.names(B)
Tbl = inner_join(B, R, by = "OTU")                                   

# Graph
Tbl.f = filter(Tbl, Bare_baseMean > 20 , TRUE)
p.72 = ggplot(Tbl.f, aes(x = log(Bare_baseMean), y = log(Rhiz_baseMean))) +
    geom_point()+
    stat_smooth(method = "lm") +
ggtitle("Day 72 vs 72")


In [ ]:
%%R
#join results tables in a single table
R = R.79
B = B.79
colnames(R) = paste("Rhiz_", colnames(R), sep ="")
colnames(B) = paste("Bare_", colnames(B), sep ="")
R = as.data.frame(R[,c(1,2,5,6)])
B= as.data.frame(B[,c(1,2,5,6)])
R$OTU = row.names(R)
B$OTU = row.names(B)
Tbl = inner_join(B, R, by = "OTU")                                   

# Graph
Tbl.f = filter(Tbl, Bare_baseMean > 20 , TRUE)
p.79 = ggplot(Tbl.f, aes(x = Bare_log2FoldChange, y = Rhiz_log2FoldChange)) +
    geom_point()+
    stat_smooth(method = "lm") +
ggtitle("Day 72 vs 79")


In [ ]:
%%R
# #join results tables in a single table
# R = R.84
# B = B.84
# colnames(R) = paste("Rhiz_", colnames(R), sep ="")
# colnames(B) = paste("Bare_", colnames(B), sep ="")
# R = as.data.frame(R[,c(1,2,5,6)])
# B= as.data.frame(B[,c(1,2,5,6)])
# R$OTU = row.names(R)
# B$OTU = row.names(B)
# Tbl = inner_join(B, R, by = "OTU")                                   

# # Graph
# Tbl.f = filter(Tbl, Bare_baseMean > 2 , TRUE)
# p.84 = ggplot(Tbl.f, aes(x = Bare_log2FoldChange, y = Rhiz_log2FoldChange)) +
#     geom_point()+
#     stat_smooth(method = "lm") +
#     ggtitle("Day 72 vs 84 \n Warning Different Libraries")


In [ ]:
%%R
# #join results tables in a single table
# R = R.88
# B = B.88
# colnames(R) = paste("Rhiz_", colnames(R), sep ="")
# colnames(B) = paste("Bare_", colnames(B), sep ="")
# R = as.data.frame(R[,c(1,2,5,6)])
# B= as.data.frame(B[,c(1,2,5,6)])
# R$OTU = row.names(R)
# B$OTU = row.names(B)
# Tbl = inner_join(B, R, by = "OTU")                                   

# # Graph
# Tbl.f = filter(Tbl, Bare_baseMean > 2 , TRUE)
# p.88 = ggplot(Tbl.f, aes(x = Bare_log2FoldChange, y = Rhiz_log2FoldChange)) +
#     geom_point()+
#     stat_smooth(method = "lm") +
#     theme(legend.position = "none") +
#     ggtitle("Day 72 vs 88 \n Warning Different Libraries")


In [ ]:
%%R -w 900 -h 900
grid.arrange(p.36, p.53, p.57, p.61, p.72, p.79, ncol = 3)

## Use responders to create a heatmap

In [ ]:
%%R
sample_data(physeq.Resp) %>% colnames

In [ ]:
%%R
gpt <- prune_taxa(names(sort(taxa_sums(physeq.Resp),TRUE)[1:300]), physeq.Resp)
plot_heatmap(gpt, sample.label="PlantSample")
plot_heatmap(gpt)